<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This script responds to challenge 1 to 4 proposed for assessment in module #5-real-time-data:** Producer, read Stream and write Stream. Also, have some Reports on data Streaming.

In [66]:
# Guarantee that don't have any other data on content folder
!rm -rf content/lake

# Setting up PySpark

In [67]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [68]:
%pip install faker

# Producer

In [69]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream
         .writeStream
         .outputMode('append')
         .option('checkpointLocation', "content/lake/bronze/checkpoint")
         .trigger(processingTime='1 seconds')
         .foreachBatch(insert_messages)
         .start()
)

query.awaitTermination(60)


False

In [70]:
query.isActive

True

In [71]:
query.stop()

In [72]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/data/*")
df.show()
df.count()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-11 20:00:...|   25|  RECEIVED|7d9473d9-4efa-4bc...|  EMAIL|      2008|   1046|
|2024-12-11 20:00:...|    2|  RECEIVED|3ba5576b-5951-48a...|  EMAIL|      2012|   1019|
|2024-12-11 20:01:...|   75|  RECEIVED|c1b12f1d-8801-488...|  EMAIL|      2004|   1039|
|2024-12-11 20:01:...|   45|  RECEIVED|bf02acae-ecf5-487...|  EMAIL|      2013|   1042|
|2024-12-11 20:01:...|   48|  RECEIVED|eafbc164-f6b0-44a...|  EMAIL|      2015|   1022|
|2024-12-11 20:00:...|   27|  RECEIVED|bf02acae-ecf5-487...|  EMAIL|      2001|   1000|
|2024-12-11 20:00:...|   34|  RECEIVED|4b700a25-f2f4-423...|  EMAIL|      2007|   1031|
|2024-12-11 20:01:...|   76|  RECEIVED|deb0876d-f6f5-4f8...|   PUSH|      2010|   1033|
|2024-12-11 20:01:...|   85|   C

103

# Additional datasets

In [73]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

In [76]:
# apply normal dataframe operations
df_stream.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



# Streaming Messages x Messages Corrupted

In [77]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

def cleansing_silver(df: DataFrame, batch_id):
    if df.isEmpty():
        print("No data in the current batch.")
        return

    # Join with countries
    df = df.join(countries, "country_id", "left")

    # Filtering corrupted messages
    corrupted_df = df.filter((col("event_type").isNull() |
                              (col("event_type") == "") |
                              (col("event_type") == "NONE")
                              ))

    # Saving corrupted messages
    corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages_corrupted/data")

    # Creating dataframe without corrupted messages
    clean_df = df.filter(~(col("event_type").isNull() |
                          (col("event_type") == "") |
                          (col("event_type") == "NONE"))
                        )
    # Saving clean messages
    clean_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages/data")


print(f'Define streaming schema...')
messages_schema = StructType([
          StructField('timestamp', TimestampType(), True),
          StructField('value', LongType(), True),
          StructField('event_type', StringType(), True),
          StructField('message_id', StringType(), True),
          StructField('channel', StringType(), True),
          StructField('country_id', IntegerType(), True),
          StructField('user_id', IntegerType(), True)
          ])

print(f'Read the streaming data...')
stream_silver_data = spark.readStream.format("parquet").schema(messages_schema).load("content/lake/bronze/messages/data/*")

print(f'Create a new date column that will be the split column in writeStreaming...')
stream_silver_data = stream_silver_data.withColumn("date", col('timestamp').cast("date"))

print(f'Write Streaming...')
stream_silver_query = (stream_silver_data
                          .writeStream
                          .outputMode('append')
                          .option('checkpointLocation', 'content/lake/silver/checkpoint')
                          .trigger(processingTime='5 seconds')
                          .foreachBatch(cleansing_silver)
                          .start()
                          )

stream_silver_query.awaitTermination(20)

Define streaming schema...
Read the streaming data...
Create a new date column that will be the split column in writeStreaming...
Write Streaming...


False

In [78]:
print(stream_silver_query.isActive)
print(stream_silver_query.stop())

True
None


## Checking data

In [79]:
def checking_silver_data():
    bronze_count = spark.read.format("parquet").load("content/lake/bronze/messages/data/*").count()
    clean_count = spark.read.format("parquet").load("content/lake/silver/messages/data/*").count()
    corrupted_count = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data/*").count()

    assert bronze_count == clean_count + corrupted_count, "Dataframes doesn't matches. Did you run the code more than once? Be careful with append mode"
    print(f'Validation passed: Bronze [{bronze_count}] = Clean [{clean_count}] + Corrupted [{corrupted_count}]')

checking_silver_data()

Validation passed: Bronze [103] = Clean [69] + Corrupted [34]


# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [80]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

In [81]:
dedup.limit(3).show()

+----------+--------------------+-----+----------+--------------------+-------+-------+-------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------+----------+
|      2012|2024-12-11 20:02:...|   98|      SENT|007b778b-3b9f-458...|   PUSH|   1026|  India|2024-12-11|
|      2000|2024-12-11 20:01:...|   60|      OPEN|07b2bfd0-da96-4c4...|   CHAT|   1045| Brazil|2024-12-11|
|      2005|2024-12-11 20:00:...|   11|  RECEIVED|07b2bfd0-da96-4c4...|   PUSH|   1000|  Italy|2024-12-11|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------+----------+



### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [82]:
print(f'Obtaining the pivot and expected schema')
df.groupBy("date", "channel").pivot("event_type").agg(count("*").alias("event_count")).fillna(0).show()

Obtaining the pivot and expected schema
+----------+-------+-------+-------+----+--------+----+
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-11|   PUSH|      4|      4|   3|       2|   4|
|2024-12-11|    SMS|      3|      1|   3|       1|   2|
|2024-12-11|  EMAIL|      2|      5|   6|       7|   3|
|2024-12-11|  OTHER|      3|      1|   1|       0|   2|
|2024-12-11|   CHAT|      2|      0|   2|       4|   4|
+----------+-------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [83]:
agg_int_total = df.groupBy("user_id").agg(count("*").alias("iterations")).fillna(0)
agg_channel_user = df.groupBy("user_id").pivot("channel").agg(count("*").alias("iterations")).fillna(0)
agg_int_total.join(agg_channel_user, "user_id", "left").sort(desc("iterations")).limit(4).show()

+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1036|         4|   1|    1|    0|   1|  1|
|   1020|         3|   1|    1|    1|   0|  0|
|   1029|         3|   1|    0|    0|   2|  0|
|   1010|         3|   0|    2|    0|   1|  0|
+-------+----------+----+-----+-----+----+---+



# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

